# Part 1

The high-speed train leaves the forest and quickly carries you south. You can even see a desert in the distance! Since you have some spare time, you might as well see if there was anything interesting in the image the Mythical Information Bureau satellite captured.

After decoding the satellite messages, you discover that the data actually contains many small images created by the satellite's camera array. The camera array consists of many cameras; rather than produce a single square image, they produce many smaller square image tiles that need to be reassembled back into a single image.

Each camera in the camera array returns a single monochrome image tile with a random unique ID number. The tiles (your puzzle input) arrived in a random order.

Worse yet, the camera array appears to be malfunctioning: each image tile has been rotated and flipped to a random orientation. Your first task is to reassemble the original image by orienting the tiles so they fit together.

To show how the tiles should be reassembled, each tile's image data includes a border that should line up exactly with its adjacent tiles. All tiles have this border, and the border lines up exactly when the tiles are both oriented correctly. Tiles at the edge of the image also have this border, but the outermost edges won't line up with any other tiles.

For example, suppose you have the following nine tiles:
```
Tile 2311:
..##.#..#.
##..#.....
#...##..#.
####.#...#
##.##.###.
##...#.###
.#.#.#..##
..#....#..
###...#.#.
..###..###

Tile 1951:
#.##...##.
#.####...#
.....#..##
#...######
.##.#....#
.###.#####
###.##.##.
.###....#.
..#.#..#.#
#...##.#..

Tile 1171:
####...##.
#..##.#..#
##.#..#.#.
.###.####.
..###.####
.##....##.
.#...####.
#.##.####.
####..#...
.....##...

Tile 1427:
###.##.#..
.#..#.##..
.#.##.#..#
#.#.#.##.#
....#...##
...##..##.
...#.#####
.#.####.#.
..#..###.#
..##.#..#.

Tile 1489:
##.#.#....
..##...#..
.##..##...
..#...#...
#####...#.
#..#.#.#.#
...#.#.#..
##.#...##.
..##.##.##
###.##.#..

Tile 2473:
#....####.
#..#.##...
#.##..#...
######.#.#
.#...#.#.#
.#########
.###.#..#.
########.#
##...##.#.
..###.#.#.

Tile 2971:
..#.#....#
#...###...
#.#.###...
##.##..#..
.#####..##
.#..####.#
#..#.#..#.
..####.###
..#.#.###.
...#.#.#.#

Tile 2729:
...#.#.#.#
####.#....
..#.#.....
....#..#.#
.##..##.#.
.#.####...
####.#.#..
##.####...
##..#.##..
#.##...##.

Tile 3079:
#.#.#####.
.#..######
..#.......
######....
####.#..#.
.#...#.##.
#.#####.##
..#.###...
..#.......
..#.###...
```

By rotating, flipping, and rearranging them, you can find a square arrangement that causes all adjacent borders to line up:
```
#...##.#.. ..###..### #.#.#####.
..#.#..#.# ###...#.#. .#..######
.###....#. ..#....#.. ..#.......
###.##.##. .#.#.#..## ######....
.###.##### ##...#.### ####.#..#.
.##.#....# ##.##.###. .#...#.##.
#...###### ####.#...# #.#####.##
.....#..## #...##..#. ..#.###...
#.####...# ##..#..... ..#.......
#.##...##. ..##.#..#. ..#.###...

#.##...##. ..##.#..#. ..#.###...
##..#.##.. ..#..###.# ##.##....#
##.####... .#.####.#. ..#.###..#
####.#.#.. ...#.##### ###.#..###
.#.####... ...##..##. .######.##
.##..##.#. ....#...## #.#.#.#...
....#..#.# #.#.#.##.# #.###.###.
..#.#..... .#.##.#..# #.###.##..
####.#.... .#..#.##.. .######...
...#.#.#.# ###.##.#.. .##...####

...#.#.#.# ###.##.#.. .##...####
..#.#.###. ..##.##.## #..#.##..#
..####.### ##.#...##. .#.#..#.##
#..#.#..#. ...#.#.#.. .####.###.
.#..####.# #..#.#.#.# ####.###..
.#####..## #####...#. .##....##.
##.##..#.. ..#...#... .####...#.
#.#.###... .##..##... .####.##.#
#...###... ..##...#.. ...#..####
..#.#....# ##.#.#.... ...##.....
```

For reference, the IDs of the above tiles are:
```
1951    2311    3079
2729    1427    2473
2971    1489    1171
```
To check that you've assembled the image correctly, multiply the IDs of the four corner tiles together. If you do this with the assembled tiles from the example above, you get 1951 * 3079 * 2971 * 1171 = 20899048083289.

Assemble the tiles into an image. What do you get if you multiply together the IDs of the four corner tiles?

In [1]:
from collections import defaultdict
from enum import Enum
from functools import reduce
from operator import mul
import re


class Direction(Enum):
    N = (0, 1)
    E = (1, 0)
    S = (0, -1)
    W = (-1, 0)
    
    
class Tile:
    def __init__(self, tile):
        self.tile = [list(row) for row in tile]
    
    def turn_to_match(self, other_tile):
        '''Rotates and possibly flips the tile to fit `other_tile`.
        
        If it is not possible it does nothing and returns None.
        Else it returns coordinate of the tile relative to `other_tile` when fit together.
        '''
        m = self.find_matching_side(other_tile)
        if m is None:
            return
        self.turn_side_to_match(**m)
        return m['to_match']
        
    def find_matching_side(self, other_tile):
        for my_dir, my_side in self.sides.items():
            for other_dir, other_side in other_tile.sides.items():
                if my_side == other_side:
                    return {'side': my_dir, 'to_match': other_dir, 'flip': False}
                if my_side == list(reversed(other_side)):
                    return {'side': my_dir, 'to_match': other_dir, 'flip': True}

    @property
    def sides(self):
        return {
            Direction.N: self.tile[0],
            Direction.S: self.tile[-1][::-1],
            Direction.W: [row[0] for row in self.tile[::-1]],
            Direction.E: [row[-1] for row in self.tile]
        }

    def turn_side_to_match(self, side, to_match, flip=False):
        '''Rotates and optionally flips the tile so its `side` side faces in the OPPOSITE direction as
        `to_match`.
        
        E.g. if `against` is 'N' then the side will be turned and/or flipped to be the "S" side of the tile.
        This is because this method is used to turn the tile to fit another one and if we want to match the
        side of this tile with the "N" side of the other tile, the side needs to be turned to "S".
        '''
        dir_order = ['N', 'E', 'S', 'W']
        turns_right = (dir_order.index(side.name) - dir_order.index(to_match.name)) % 4
        self.rotate(turns_right)
        if flip:
            axis = 'x' if to_match in (Direction.E, Direction.W) else 'y'
            self.flip(axis)

    def flip(self, axis='y'):
        if axis == 'y':
            self.tile = [list(reversed(row)) for row in self.tile]
        elif axis == 'x':
            self.tile = list(reversed(self.tile))
        else:
            raise Exception(
                'Unsupported `axis` value for `Tile.flip`: %s. Should be either "x" or "y".' % axis)
    
    def rotate(self, turns_right=1):
        turns_right %= 4
        for t in range(turns_right):
            cols = [[row[i] for row in self.tile] for i in range(len(self.tile[0]))]
            self.tile = [list(reversed(c)) for c in cols]

            
# Test
tile1 = Tile(['123', '456'])
tile2 = Tile(['14', '78'])
for i in range(4):
    m = tile2.find_matching_side(tile1)
    assert m is not None and m['to_match'] is Direction.W and m['flip'] is True
    tile2.rotate()
tile2.flip()
for i in range(4):    
    m = tile2.find_matching_side(tile1)
    assert m is not None and m['to_match'] is Direction.W and m['flip'] is False
    tile2.rotate()
tile2.flip()
tile1.turn_to_match(tile2)
assert tile2.sides[Direction.N] == tile1.sides[Direction.S]


class InputParser:
    def __init__(self):
        self.tiles = {}
        self._current_id = None
        self._current_tile = []

    def parse_line(self, line):
        line = line.rstrip()
        m = re.match(r'Tile (?P<tile_id>\d+):', line)
        if m is not None:
            tile_id = m.group('tile_id')
            self._current_id = tile_id
        elif line == '':
            self._commit_current_tile()
        else:
            self._current_tile.append(line)
            
    def _commit_current_tile(self):
        if len(self._current_tile) > 0:
            self.tiles[self._current_id] = Tile(self._current_tile)
            self._current_tile = []
            self._current_id = None    

    def result(self):
        self._commit_current_tile()
        return self.tiles
        
        
def get_input(parser=None):
    if parser is None:
        parser = InputParser()
    with open('inputs/20') as f:
        for line in f.readlines():
            parser.parse_line(line)
    return parser.result()


def find_possible_neighbors(to_id, tiles):
    to_tile = tiles[to_id]
    poss_neighbors = defaultdict(set)
    for poss_neigh_id, poss_neigh_tile in tiles.items():
        if poss_neigh_id == to_id:
            continue
        m = poss_neigh_tile.find_matching_side(to_tile)
        if m is not None:
            poss_neighbors[m['to_match']].add(poss_neigh_id)
    return poss_neighbors
    

def find_tiles_with_n_neighbors(tiles, n):
    'Tiles with 2 neighbors are sure to be corners and those with 2 or 3 makes frame.'
    result = set()
    for t_id, tile in tiles.items():
        poss_neighs = find_possible_neighbors(t_id, tiles)
        if sum(len(poss_neighs_in_dir) > 0 for poss_neighs_in_dir in poss_neighs.values()) == n:
            result.add(t_id)
    return result


def result1():
    tiles = get_input()
    corners =  [int(i) for i in find_tiles_with_n_neighbors(tiles, 2)]
    assert len(corners) == 4, 'Some corners have more than 2 possible neighbors'
    return reduce(mul, corners)

    
result1()

22878471088273